In [10]:
print("Hello...")

Hello...


### Import

In [11]:
import boto3 
import pandas as pd 
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession 

s3_client = boto3.resource('s3') 
pipeline_name = f"emp-bonus-training-pipeline"
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
pipeline_session = PipelineSession()
default_bucket = sagemaker_session.default_bucket()
model_package_group_name = f"EmpBonusPackageGroup"

print(f"Default S3 Bucket Name: {default_bucket}")
print(f"Model Package Group Name: {model_package_group_name}")

Default S3 Bucket Name: sagemaker-us-east-1-654654157683
Model Package Group Name: EmpBonusPackageGroup


In [12]:
from sagemaker.workflow.parameters import ( 
 ParameterInteger, 
 ParameterString, 
 ParameterFloat) 

base_job_prefix = "emp-bonus"
processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
processing_instance_type = ParameterString( name="ProcessingInstanceType", default_value="ml.m5.xlarge") 
training_instance_type = ParameterString( name="TrainingInstanceType", default_value="ml.m5.xlarge") 
input_data = "data/mock_data.csv" 
model_approval_status = ParameterString( name="ModelApprovalStatus", default_value="PendingManualApproval")

### Preprocessing

In [13]:
# Define Processing Step for Feature Engineering
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

framework_version = "1.0-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name="emp-pre-processing",
    role=role,
    sagemaker_session=pipeline_session,
)

processor_args = sklearn_processor.run(
    inputs=[
      ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),  
    ],
    outputs=[
        ProcessingOutput(output_name="processed-data", source='/opt/ml/processing/output',
                         destination=f"s3://{default_bucket}/output/processed")
    ],
    code=f"preprocessing_script.py",
)
step_preprocess = ProcessingStep(name="EmpBonusPreProcessing", step_args=processor_args)

The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is interpreted in pipeline execution time only. As the function needs to evaluate the argument value in SDK compile time, the default_value of this Parameter object will be used to override it. Please make sure the default_value is valid.
/opt/conda/lib/python3.12/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


### Step 5: Model Training

In [14]:
# Model Traning Step

input_data = f"s3://{default_bucket}/output/processed/transformed_data.csv"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name="emp-data-split",
    role=role,
    sagemaker_session=pipeline_session,
)

processor_args = sklearn_processor.run(
    inputs=[
      ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),  
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train",
                         destination=f"s3://{default_bucket}/output/train" ),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation",
                         destination=f"s3://{default_bucket}/output/validation"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test",
                         destination=f"s3://{default_bucket}/output/test")
    ],
    code=f"model_training_script.py",
)
step_data_split = ProcessingStep(name="EmpBonusDataSplit", step_args=processor_args, depends_on=[step_preprocess] )

The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is interpreted in pipeline execution time only. As the function needs to evaluate the argument value in SDK compile time, the default_value of this Parameter object will be used to override it. Please make sure the default_value is valid.


### Model Train

In [15]:
print("Model Traing Job")
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep

# Specify the container image for Linear Learner (built-in algorithm)
linear_learner_container = sagemaker.image_uris.retrieve('linear-learner', sagemaker_session.boto_region_name)

# Define the LinearLearner estimator
linear_estimator = Estimator(
    image_uri=linear_learner_container,
    role=role,
    instance_count=1,
    instance_type=training_instance_type,
    output_path=f's3://{default_bucket}/model-output',
    sagemaker_session=pipeline_session,
    base_job_name="emp-bonus-linear-learner"
)

# Set hyperparameters for Linear Learner
linear_estimator.set_hyperparameters(
    predictor_type='regressor',  # We're solving a regression problem
    mini_batch_size=32,
    epochs=10
)

train_path = f"s3://{default_bucket}/output/train/train.csv"
val_path = f"s3://{default_bucket}/output/validation/validation.csv"

from sagemaker.inputs import TrainingInput

# Specify the input data channels
train_input = TrainingInput(
    s3_data=train_path, 
    content_type='text/csv'
)
val_input = TrainingInput(
    s3_data=val_path, 
    content_type='text/csv'
)

# Training Step
step_training = TrainingStep(
    name="EmpBonusTrainingStep",
    estimator=linear_estimator,
    inputs={
        'train': train_input,
        'validation': val_input
    },
    depends_on=[step_data_split]
)


Model Traing Job


### Model Register

In [17]:
import sagemaker
from sagemaker import Model
from sagemaker.workflow.model_step import ModelStep
from sagemaker.model_metrics import MetricsSource, ModelMetrics

# Determine what AWS region we are in
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name 
# Retrieve the Linear Learner image URI
container_image_uri = sagemaker.image_uris.retrieve(framework="linear-learner", region=region)

model = Model(
    image_uri=container_image_uri,
    model_data=step_training.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=pipeline_session,
    role=role,
)

model_approval_status = "PendingManualApproval"
customer_metadata_properties = {"ModelType": "EmpBonusPrediction"}

register_args = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    customer_metadata_properties=customer_metadata_properties,
)

step_register = ModelStep(name="EmpBonusRegisterModel", step_args=register_args)

/opt/conda/lib/python3.12/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [18]:
import json
from sagemaker.workflow.pipeline import Pipeline

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_count,
        processing_instance_type,
        training_instance_type,
        model_approval_status,
        input_data,
    ],
    steps=[step_preprocess, step_data_split, step_training, step_register],
)
definition = json.loads(pipeline.definition())
# print(definition)

In [19]:
# Create a new or update existing Pipeline
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:654654157683:pipeline/emp-bonus-training-pipeline',
 'ResponseMetadata': {'RequestId': '56dfab9a-e804-456c-8757-eca83789aa80',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '56dfab9a-e804-456c-8757-eca83789aa80',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '95',
   'date': 'Sun, 03 Aug 2025 18:18:52 GMT'},
  'RetryAttempts': 0}}

In [20]:
# start Pipeline execution
pipeline.start()

_PipelineExecution(arn='arn:aws:sagemaker:us-east-1:654654157683:pipeline/emp-bonus-training-pipeline/execution/t0cat13b9mva', sagemaker_session=<sagemaker.session.Session object at 0x7fbadf60be00>)